In [50]:
import os
import datetime
from datetime import timedelta
import csv
import time
import re
import string
import requests
import numpy as np
from urllib.parse import urlencode
import selenium
from selenium                         import webdriver
from selenium.webdriver import Firefox
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by  import   By 
from selenium.webdriver.firefox.options  import   Options 
from selenium.webdriver.support import   expected_conditions  as   EC 
from selenium.webdriver.support.wait  import   WebDriverWait
from selenium.common.exceptions       import NoSuchElementException as NSE #gestione delle eccezioni
from selenium.common.exceptions       import TimeoutException as TOE
from selenium.common.exceptions       import StaleElementReferenceException as SERE
from selenium.common.exceptions       import ElementClickInterceptedException as ECIE
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary

In [51]:
# COSTANTI 
italia=[{'regione':'Abruzzo',               'provincie': ['L\'Aquila','Teramo','Pescara','Chieti']},
        {'regione':'Basilicata',            'provincie': ['Potenza','Matera']},
        {'regione':'Calabria',              'provincie': ['Cosenza','Catanzaro','Crotone','Vibo Valentia','Reggio Calabria']},
        {'regione':'Campania',              'provincie': ['Caserta','Benevento','Avellino','Salerno','Napoli']},
        {'regione':'Emilia-Romagna',        'provincie': ['Piacenza','Parma','Reggio Emilia','Modena','Ferrara','Ravenna','Bologna','Rimini','Forlì']},
        {'regione':'Friuli Venezia Giulia', 'provincie': ['Udine','Gorizia','Trieste','Pordenone']},
        {'regione':'Lazio',                 'provincie': ['Viterbo','Rieti','Latina','Frosinone','Roma']},
        {'regione':'Liguria',               'provincie': ['Imperia','Savona','La Spezia','Genova']},
        {'regione':'Lombardia',             'provincie': ['Bergamo','Brescia','Como','Cremona','Lecco','Lodi','Mantova','Milano','Varese','Monza','Sondrio','Pavia']},
        {'regione':'Marche',                'provincie': ['Ancona','Macerata','Ascoli Piceno','Fermo','Pesaro']},
        {'regione':'Molise',                'provincie': ['Campobasso','Isernia']},
        {'regione':'Piemonte',              'provincie': ['Asti','Alessandria','Biella','Vercelli','Novara','Cuneo','Torino','Verbania']},
        {'regione':'Puglia',                'provincie': ['Foggia','Taranto','Brindisi','Lecce','Bari','Andria']},
        {'regione':'Sardegna',              'provincie': ['Sassari','Nuoro','Oristano','Cagliari','Carbonia']},
        {'regione':'Sicilia',               'provincie': ['Trapani','Agrigento','Caltanissetta','Enna','Ragusa','Siracusa','Palermo','Messina','Catania']},
        {'regione':'Toscana',               'provincie': ['Lucca','Pistoia','Livorno','Pisa','Arezzo','Siena','Grosseto','Prato','Firenze','Massa']},
        {'regione':'Trentino-Alto Adige',   'provincie': ['Bolzano','Trento']},
        {'regione':'Umbria',                'provincie': ['Perugia','Terni']},
        {'regione':'Valle d\'Aosta',        'provincie': ['Aosta']},
        {'regione':'Veneto',                'provincie': ['Verona','Vicenza','Belluno','Treviso','Padova','Rovigo','Venezia']}  
]

#cities=['Catanzaro, Calabria, Italia','Crotone']
attesaCorta=3 #variabile attesa esplicita caricamento pagina
#ignoredExceptions=(NSE, SERE, TOE, ECIE)

In [52]:
def preparaDriver():
    options = Options()
    options.headless = True  #modalita senza interfaccia grafica con false
    options.add_argument('--lang=it') #imposto la lingua del browser in italiano
    options.set_preference("permissions.default.image", 2) #disablita lo scaricamento delle immagini
    driver = Firefox(options=options) #invoco il webdriver di firefox geckodriver in path
    return driver


def scrapeNumeroStruttureDisponibili(driver, dataOdierna, regione, numstruttureTrovate):  #visita le singole pagine degli hotel 
    hotel=dict()
    hotel['regione']=regione
    hotel['citta']=stringaCitta.split(",")[0]  #nome della città
    hotel['data odierna']=dataOdierna
    hotel['disponibilita odierna']=numstruttureTrovate
    print(numstruttureTrovate)
    return hotel

if __name__ == '__main__':
#def main():
    try:
        alloggiCitta=list()
        date = datetime.date.today() #giorno attuale
        driver = preparaDriver()
        dataOdierna=date.strftime("%d/%m/%Y")
        for regione in italia:
            nomeRegione = regione['regione']
            for city in regione['provincie']:
                print(nomeRegione+"-"+city)
                parametri = { # parametri vuoti per prendere le strutture totali disponibili indipendentemente dal giorno
                            'ss': '',
                            'dest_type': 'city'
                }
                parametri['ss'] = city+", "+nomeRegione+", Italia"
                queryString = urlencode(parametri) #libreria urlLib per formattare la url
                url="https://www.booking.com/searchresults.html?%s" % queryString
                driver.get(url)
                #driver = preparaDriver(url)
                stringaCitta=city #variabile da assegnare alla lista dei campi_alloggi
                valoreRegione=regione.values()
                iteratoreRegioni=iter(valoreRegione)
                primoValore=next(iteratoreRegioni)
                stringaRegione=primoValore
                xpathStruttura = driver.find_element_by_xpath('//*[@id="right"]/div[5]/div/div[1]/div/h1').text  #percorso del titolo che varia a seconda delle citta
                numstruttureTrovate=(re.findall('[\d]*[\.]?[\d]+',xpathStruttura)[0]) #prende il primo numero in lista relativo alle strutture
               
                wait = WebDriverWait(driver, attesaCorta).until( #attesa esplicita
                    EC.presence_of_all_elements_located(
                        (By.CLASS_NAME, 'sr-hotel__title')))
                alloggiCitta.append(scrapeNumeroStruttureDisponibili(driver, dataOdierna, stringaRegione, numstruttureTrovate))
                
        keys = alloggiCitta[0].keys()
        with open(dataOdierna.replace('/','-')+'_disponibilitaOdiernaBooking.csv', 'a', newline='') as csvFile: #appende ai dati precedenti 
            dict_writer = csv.DictWriter(csvFile, keys) # imposto i nomi dei campi
            dict_writer.writeheader()
            dict_writer.writerows(alloggiCitta)
    
    finally:
        driver.close()
        driver.quit()

        

Abruzzo-L'Aquila
82
Abruzzo-Teramo
20
Abruzzo-Pescara
242
Abruzzo-Chieti
60
Basilicata-Potenza
65
Basilicata-Matera
716
Calabria-Cosenza
71
Calabria-Catanzaro
57
Calabria-Crotone
125
Calabria-Vibo Valentia
35
Calabria-Reggio Calabria
224
Campania-Caserta
161
Campania-Benevento
64
Campania-Avellino
23
Campania-Salerno
681
Campania-Napoli
4.005
Emilia-Romagna-Piacenza
54
Emilia-Romagna-Parma
251
Emilia-Romagna-Reggio Emilia
71
Emilia-Romagna-Modena
125
Emilia-Romagna-Ferrara
221
Emilia-Romagna-Ravenna
158
Emilia-Romagna-Bologna
1.426
Emilia-Romagna-Rimini
1.033
Emilia-Romagna-Forlì
45
Friuli Venezia Giulia-Udine
85
Friuli Venezia Giulia-Gorizia
28
Friuli Venezia Giulia-Trieste
516
Friuli Venezia Giulia-Pordenone
29
Lazio-Viterbo
207
Lazio-Rieti
40
Lazio-Latina
34
Lazio-Frosinone
25
Lazio-Roma
10.722
Liguria-Imperia
299
Liguria-Savona
112
Liguria-La Spezia
822
Liguria-Genova
876
Lombardia-Bergamo
431
Lombardia-Brescia
122
Lombardia-Como
546
Lombardia-Cremona
58
Lombardia-Lecco
105
Lombard

In [30]:
print(regione)

{'regione': 'Abruzzo', 'provincie': ["L'Aquila", 'Teramo', 'Pescara', 'Chieti']}


In [31]:
print(regione.keys())

dict_keys(['regione', 'provincie'])


In [34]:
print(regione.values())

dict_values(['Abruzzo', ["L'Aquila", 'Teramo', 'Pescara', 'Chieti']])


In [36]:
reg=regione.values()

In [37]:
value_iterator = iter(reg)

In [38]:
first_value = next(value_iterator)